In [6]:
# imports
import os
import sys
from typing import List, Tuple
from tqdm import tqdm

sys.path.append(os.path.dirname(os.getcwd()))

import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torchvision.models.detection.roi_heads import maskrcnn_inference

from src.model.engine import FitterMaskRCNN
from src.model.model import maskRCNNModel, predict_image
from src.model.dataset import InferenceMaskRCNNDataset, MaskRCNNDataset
from src.utils.utils import *
from src.utils.const import *


In [7]:
dataset_1 = "../dataset/train"

dataset = MaskRCNNDataset(dataset_path=dataset_1, datatype="train")

# load the model
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu") 
model = maskRCNNModel()
model_weights_path = os.path.join(os.path.dirname(os.getcwd()), "model/best-checkpoint-0.bin")
checkpoint = torch.load(model_weights_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


/var/folders/74/pjwdjs550d91v4lc_1_rqm6r0000gn/T/ipykernel_14840/3354357200.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_weights_path, m

In [8]:
model.train()
images = []
targets = []
for i in range(20):
    image, target = dataset[i]
    images.append(image)
    targets.append(target)
pred = model(images, targets)


In [5]:
optimizer.zero_grad()

In [ ]:
import matplotlib.pyplot as plt
model.eval()
for image, meta in inference_ds:
    image_norm = image.unsqueeze(0).to(device)
    with torch.no_grad():
        prediction = model(image_norm)
    print(prediction)
    break

NameError: name 'inference_ds' is not defined

In [ ]:
import matplotlib.pyplot as plt
model.eval()
for image, meta in inference_ds:
    image_norm = image.unsqueeze(0).to(device)
    image_norm, _ = model.transform(image_norm, None)
    features = model.backbone(image_norm.tensors)
    proposals, _ = model.rpn(image_norm, features)
    for box in proposals[0]:
        plt.imshow(image_norm.tensors[0].detach().cpu().numpy().transpose(1, 2, 0))
        x = [box[0], box[2], box[2], box[0], box[0]]
        y = [box[1], box[1], box[3], box[3], box[1]]   
        plt.plot(x, y, color="red")
        plt.show()
    detections, _ = model.roi_heads(features, proposals, image_norm.image_sizes)
    detections = model.transform.postprocess(detections, image_norm.image_sizes, [(meta["height"], meta["width"])])

In [ ]:
import torch
from matplotlib import patches

def reshape_annotations(annotations: pd.DataFrame, original_size: List[int], new_size: List[int]) -> torch.tensor:
    # annotations = annotations[["x1", "y1", "x2", "y2"]] # napari invers x and y coordinates
    tensor = torch.tensor(annotations.values, dtype=torch.float32)
    if tensor.shape[1] == 1:
        tensor = tensor.T
    x_reshape = new_size[0] / original_size[0]
    y_reshape = new_size[1] / original_size[1]
    tensor[:, 0] *= x_reshape
    tensor[:, 2] *= x_reshape
    tensor[:, 1] *= y_reshape
    tensor[:, 3] *= y_reshape
    return tensor


for annotations_1_path, (image, meta) in zip(annotations_1_paths, inference_ds):
    annotations_1 = pd.read_csv(annotations_1_path, index_col=0)
    if len(annotations_1) == 0:
        print(f"no annotations for {annotations_1_path}")
        continue
    image_norm = image.unsqueeze(0).to(device)
    image_norm, _ = model.transform(image_norm, None)
    t = image_norm.tensors[0]
    t =  t - t.min()
    features = model.backbone(image_norm.tensors)
    proposals = reshape_annotations(annotations_1, [meta["width"], meta["height"]], [image_norm.image_sizes[0][1], image_norm.image_sizes[0][0]])
    mask_features = model.roi_heads.mask_roi_pool(features, [proposals], image_norm.image_sizes)
    mask_features = model.roi_heads.mask_head(mask_features)
    mask_logits = model.roi_heads.mask_predictor(mask_features)
    labels = [torch.ones(len(proposals), device=device, dtype=torch.int64)]
    masks_probs = maskrcnn_inference(mask_logits, labels)
    print(proposals.shape)

    # detections, _ = model.roi_heads(features, [proposals], image_norm.image_sizes)
    detections = [{
        "boxes": proposals,
        "masks": masks_probs[0],
        "scores": torch.ones(len(proposals)),
        "labels": labels,
    }]
    
    detections = model.transform.postprocess(detections, image_norm.image_sizes, [(meta["height"], meta["width"])])
    # assert detections[0]["boxes"].shape == proposals.shape, f"{detections[0]["boxes"].shape} != {proposals.shape}"
    i = 0
    for box_pred, mask in zip(detections[0]["boxes"], detections[0]["masks"]):

        fig, axes = plt.subplots(1, 2)
        fig.set_size_inches(10, 5)

        # x = [box_or["x1"], box_or["x2"], box_or["x2"], box_or["x1"], box_or["x1"]]
        # y = [box_or["y1"], box_or["y1"], box_or["y2"], box_or["y2"], box_or["y1"]]
        # axes[0].imshow(image.detach().cpu().numpy().transpose(1, 2, 0))
        # axes[0].plot(x, y, color="red")

        box_pred = box_pred.detach().cpu().numpy()
        x = [box_pred[0], box_pred[2], box_pred[2], box_pred[0], box_pred[0]]
        y = [box_pred[1], box_pred[1], box_pred[3], box_pred[3], box_pred[1]]
        axes[0].imshow(image.detach().cpu().numpy().transpose(1, 2, 0))
        axes[0].plot(x, y, color="red")

        image_masked = image * mask
        axes[1].imshow(image_masked.detach().cpu().numpy().transpose(1, 2, 0))
        fig.suptitle(f"box {i}")
        plt.show()
        i += 1
        break
    break